M Kronig (23029039)
CSC-40098 Final Project
Analysis of posts from the CS Educators Stack Exchange website
(SS3) What questions are computer science eduators asking?

In [2]:
#Test notebook is running
print('Hello world!')

Hello world!
